In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import sys
import importlib
import pickle
import glob
import random
import os
import shutil

sys.path.append('..')

from pathlib import Path
from tqdm import tqdm
from collections import Counter

from analysis import label_tools as lt
from analysis import prepare
from analysis import visualization
from analysis import preprocessing
from analysis import features as ft
from analysis import cluster
from analysis import evaluation

from analysis.constants import RANDOM_SEEDS

importlib.reload(ft)
importlib.reload(prepare)

random.seed(698)

In [ ]:
# stadtwald
site_folder = '/home/richard/data/Bamberg_Stadtwald/test/'
orthophoto_dir = site_folder + 'gt_orthos/'
gt_mask_dir = site_folder + 'gt_masks/' # from gt species labels
prediction_dir = site_folder + 'predictions/pred_crown_tiles/' # predicted crowns

# define folders
encoding_dir = Path(site_folder + 'encodings')
label_encoding_dir = Path(site_folder + 'label_encodings')
result_dir = Path(site_folder + 'results')
Path(encoding_dir).mkdir(parents=True, exist_ok=True)
Path(label_encoding_dir).mkdir(parents=True, exist_ok=True)
Path(result_dir).mkdir(parents=True, exist_ok=True)

assert Path(site_folder).exists()
assert Path(orthophoto_dir).exists()
assert Path(gt_mask_dir).exists()
assert Path(prediction_dir).exists()
assert Path(encoding_dir).exists()
assert Path(label_encoding_dir).exists()
assert Path(result_dir).exists()

In [ ]:
# 1. extract single tree crowns (polygons) from orthophoto
if not Path(site_folder + 'pred_polygon_clipped_raster_files').exists():
    prepare.clip_crown_sets_with_gt_masks(orthophoto_dir, prediction_dir, gt_mask_dir, make_squares=False, step_size=0.5)
else:
    print("Cropped files already exists")

In [ ]:
# 2. preprocess images create two sets
second_set = preprocessing.preprocess_images(site_folder + 'pred_polygon_clipped_raster_files', None, None, False, True, False, False)

In [ ]:
# 3. Feature extraction
second_set = '/home/richard/data/Bamberg_Stadtwald/test/preprocessed1_None_clahe_clipped_pred_polygon_311/' # smaller amount of images

cnns = ['resnet']
encoding_files = sorted(encoding_dir.glob('*.pickle'))
if len(encoding_files) != 1:
    clahe = ft.create_and_save_le_encodings(cnns[0], second_set, site_folder)
else:
    print("Encodings already exists")

In [ ]:
# 5. Analysis
fc1_path = Path('/home/richard/data/Bamberg_Stadtwald/test/encodings/lower_sample/resnet_clahe_polygon_pred.pickle')
le_path = Path('/home/richard/data/Bamberg_Stadtwald/test/label_encodings/lower_sample/resnet_clahe_label_encodings.pickle')
assert fc1_path.is_file()
assert le_path.is_file()
with open(fc1_path, 'rb') as f:
    data = pickle.load(f)
with open(le_path, 'rb') as l:
    le = pickle.load(l)
files = data['filename']
fc1 = data['features']
labels = data['labels']
y_gt = le.transform(labels)

micro_f1_scores = []
macro_f1_scores = []
weighted_f1_scores = []
pred_species = []
for seed in tqdm(RANDOM_SEEDS): # 5
    reduced = cluster.umap(fc1, seed)
    y_pred, micro_f1_score, macro_f1_score, weighted_f1_score, f_star, cohen_kappa, mcc, nmi, num_pred_species = cluster.optics(reduced, y_gt, seed)
    micro_f1_scores.append(micro_f1_score)
    macro_f1_scores.append(macro_f1_score)
    weighted_f1_scores.append(weighted_f1_score)
    pred_species.append(num_pred_species)

In [ ]:
print(np.mean(micro_f1_scores), '\n', np.mean(macro_f1_scores), '\n', np.mean(weighted_f1_scores))
print(round(np.mean(pred_species)))
# lower random sample

In [ ]:
print(np.mean(micro_f1_scores), '\n', np.mean(macro_f1_scores), '\n', np.mean(weighted_f1_scores))
print(round(np.mean(pred_species)))
# large sample

In [ ]:
# copy random sample of 311 files
source = '/home/richard/data/Bamberg_Stadtwald/test/preprocessed_None_clahe_clipped_pred_polygon_858/'
destination = '/home/richard/data/Bamberg_Stadtwald/test/random_sample/'

files = glob.glob(source + '/*.png')
no_of_files = 311

for file_name in random.sample(files, no_of_files):
    shutil.move(os.path.join(source, file_name), destination)